# Мониторинг проектов

## Импорт библиотек и загрузка файлов

In [58]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
from openpyxl import load_workbook 
from openpyxl.drawing.image import Image
from datetime import date
import helpers, metrics

In [9]:
PATH = "C:\\Users\\SEA5324\\Nextcloud\\Planning Center\\100_Projects\\06 _Project_Planning Center\\Project orders reports"
os.mkdir(PATH + f"\\{date.today()}")
os.mkdir(PATH + f"\\{date.today()}\\Входные файлы")
os.mkdir(PATH + f"\\{date.today()}\\Выходные файлы")
os.mkdir(PATH + f"\\{date.today()}\\Выходные файлы\\Общие")
os.mkdir(PATH + f"\\{date.today()}\\Выходные файлы\\Бизнес")
os.mkdir(PATH + f"\\{date.today()}\\Выходные файлы\\Снабжение")

In [4]:
today_is_Tuesday = True # сформировать основной отчет + выгрузить разбивку по снабженцам в папку снабжение
today_is_Friday = False # сформировать основной отчет + подтянуть комменты снабженцев со вторника, сделать сводник и отчеты для бизнеса
comments_needed = False
if today_is_Tuesday: comments_delay = 4
if today_is_Friday: comments_delay, last_week_delay = 4, 7

In [5]:
start = datetime.now() # засечем, сколько заняла загрузка всех файлов
shipped_goods1 = pd.read_excel(PATH + f'\\{date.today()}\\Входные файлы\\Список_всех_РТиУ_с_деталями (11.09.2024-25.10.2024).xlsx', skipfooter=1)
shipped_goods2 = pd.read_excel(PATH + f'\\{date.today()}\\Входные файлы\\Список_всех_РТиУ_с_деталями (01.06.2024-20.08.2024).xlsx', skipfooter=1)
shipped_goods3 = pd.read_excel(PATH + f'\\{date.today()}\\Входные файлы\\Список_всех_РТиУ_с_деталями (21.08.2024-10.09.2024).xlsx', skipfooter=1)
shipped_goods4 = pd.read_excel(PATH + f'\\{date.today()}\\Входные файлы\\Список_всех_РТиУ_с_деталями (26.10.2024-27.06.2025).xlsx', skipfooter=1)
shipped_goods = pd.concat([shipped_goods1, shipped_goods2, shipped_goods3, shipped_goods4])
df = pd.read_excel(PATH + f'\\{date.today()}\\Входные файлы\\Заказы с обеспечением.xlsx')
scm = pd.read_excel(PATH + '\\Постоянные файлы\\SCM_report.xlsx')
projects = pd.read_excel(PATH + '\\Постоянные файлы\\Projects.xlsx')
gl = pd.read_excel(PATH + '\\Постоянные файлы\\Глоссарий.xlsx', sheet_name='Полный')
finish = datetime.now()
print("Загрузка файлов заняла ", str(finish - start))

C:\Users\SEA5324\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\SEA5324\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\SEA5324\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\SEA5324\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply o

### Data cleaning

In [29]:
# КОСМЕТИКА
# переименуем колонки для удобства
df = df.rename(columns={'Проект':'Название проекта', 'Контрагент.Код исторической системы (Склады и магазины)':"КИС", "Заказ":"Номер клиентского заказа", "Дата":"Дата заказа", "Количество (в единицах хранения)":"Количество", "Заказ на поступление.Контрагент": "Поставщик", "Заказ на поступление.Дата поступления":"Дата поступления", "Номенклатура.Продакт-менеджер":"Продакт-менеджер", "Контрагент.ИНН":"ИНН"})
df.drop(index=df[df['Номенклатура'].str.contains(".слуга", regex=True)].index, inplace=True) # убираем услуги
df.drop(index=df[df['Номенклатура'].str.contains("Штраф", regex=True)].index, inplace=True) # убираем штрафы
df.loc[:, 'Статус проектной линии'] = 'Статус отсутствует' # создаем колонку для хранения статуса и примечания
df['Дата заказа'] = pd.to_datetime(df['Дата заказа'], format='%d.%m.%Y').dt.date
df['Ключ клиентского заказа'] = df['Номер клиентского заказа'] + '&' + df['Дата заказа'].astype(str)
projects['Ключ клиентского заказа'] = projects['Номер клиентского заказа'] + '&' + projects['Дата заказа'].astype('str')
df = df.merge(projects[['Ключ клиентского заказа', 'Критическая дата', 'Конечный заказчик']], on='Ключ клиентского заказа', how='left')

df['Id строки EDI'] = df['Id строки EDI'].fillna(0)
df['Id строки EDI'] = df['Id строки EDI'].astype(int)

# уберем пропуски в Обеспечено/Не обеспечено
df['Не обеспечено'] = df['Не обеспечено'].fillna(0)
df['Обеспечено'] = df['Обеспечено'].fillna(0)
df['Обеспечено ЗП'] = df['Обеспечено ЗП'].fillna(0)

# Проверка на частичную отгрузку 
df.loc[(df['Вариант обеспечения'] != 'Отгрузить') & (df['Заказ на поступление'].str.contains('еремещ')) & (df['Обеспечено ЗП'] < df['Обеспечено']) & (df['Обеспечено ЗП'] != 0), 'Обеспечено'] = df.loc[(df['Вариант обеспечения'] != 'Отгрузить') & (df['Обеспечено ЗП'] < df['Обеспечено']) & (df['Обеспечено ЗП'] != 0), 'Обеспечено ЗП']
df.loc[(df['Обеспечено'] < df['Количество (в единицах хранения)']) & (df['Обеспечено'] > 0) & (df['Отменено'] == 'Нет'), 'Частично обеспечено'] = 'Да'
df.loc[(df['Не обеспечено'] < df['Количество (в единицах хранения)']) & (df['Не обеспечено'] > 0) & (df['Отменено'] == 'Нет'), 'Частично не обеспечено'] = 'Да'

df.loc[(df['Частично обеспечено'] == 'Да'), 'Количество (в единицах хранения)'] = df[df['Частично обеспечено'] == 'Да']['Обеспечено']
df.loc[(df['Частично не обеспечено'] == 'Да'), 'Количество (в единицах хранения)'] = df[df['Частично не обеспечено'] == 'Да']['Не обеспечено']

# заменим кол-ва в заказе на кол-ва к обеспечению для некоторых строк (которые не в отгрузке и к которым есть строки к обеспечению)
df['Цена с НДС'] = df['Сумма с НДС'] / df['Количество (в единицах хранения)']
# заменим кол-ва в заказе на кол-ва к обеспечени для некоторых строк (которые не в отгрузке и к которым есть строки к обеспечению)
df['Стоимость с НДС'] = df['Цена с НДС'] * df['Количество (в единицах хранения)']
df.drop(['Сумма с НДС'], inplace=True, axis=1) # уберем колонку, чтобы не путать со стоимостью с НДС

# обработка файла SCM

# сбытовой статус артикула зависит еще от того, к какому складу привязан заказ
# сокращаем отчет scm только для материалов по проекту и обрабатываем столбец склад, статус продаж/закупок
scm['Склад'] = scm['Склад'].apply(lambda x: x[:9])
df['Склад'] = df['Склад'].apply(lambda x: x[:9])
scm = scm[(scm['Артикул'].isin(df['Артикул'].unique())) & (scm['Склад'].isin(df['Склад'].unique()))]
scm['Статус продаж'] = scm[~scm['Статус продаж'].isna()]['Статус продаж'].apply(lambda x: x[:2])
scm['Статус закупок'] = scm[~scm['Статус закупок'].isna()]['Статус закупок'].apply(lambda x: x[:2])

# новые ключи для мэтча сбытового статуса
df['Артикул&Склад'] = df['Артикул'] + df['Склад']
scm['Артикул&Склад'] = scm['Артикул'] + scm['Склад']

# добавляем в таблицу локальный поток или нет
local_routes = ('Ms', 'RU', 'Sp')
local = scm[~scm['Маршрут'].isna()][['Артикул', 'Маршрут']]
local = local.drop_duplicates(subset='Артикул')
local.loc[local['Маршрут'].str.startswith(local_routes), 'Локальный поток ДА/НЕТ'] = 'ДА'
local.loc[~local['Маршрут'].str.startswith(local_routes), 'Локальный поток ДА/НЕТ'] = 'НЕТ'
df = df.merge(local[['Артикул', 'Локальный поток ДА/НЕТ']], on='Артикул', how='left')

# формируем таблицу с закупочным/сбытовым статусами
material_status = scm[['Артикул&Склад', 'Статус продаж', 'Статус закупок']].drop_duplicates(subset='Артикул&Склад')

# добавляем колонки в pdf - выполнять эту строку только один раз!
df = df.merge(material_status, on='Артикул&Склад', how='left')

# найдем MRP контролеров для каждого артикула
mrp_controllers_df = scm[(~scm['MRP контролер'].isna())][['Артикул', 'MRP контролер']].drop_duplicates(subset='Артикул')
df = df.merge(mrp_controllers_df, on='Артикул', how='left')

# проверка на завершенные проекты 
finished_projects = df[df['Название проекта'].isin(projects[projects['Статус проекта'] == 'Завершен']['Проект'].unique().tolist())]
df = df[df['Название проекта'].isin(projects[projects['Статус проекта'] != 'Завершен']['Проект'].unique().tolist())]

# проверка на новые проекты
new_projects = projects.loc[projects['Статус проекта'] == 'Новый']['Проект'].unique().tolist()
# для новых проектов в файле отметим дату начала отслеживания и дату первого размещенного заказа по проекту
for new_project in new_projects:
    projects.loc[projects['Проект'] == new_project, 'Дата начала отслеживания'] = date.today()
    projects.loc[projects['Проект'] == new_project, 'Дата первого заказа проекта'] = projects[projects['Проект'] == new_project]['Дата заказа'].min()

C:\Users\SEA5324\AppData\Local\Temp\ipykernel_24388\1336388840.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2025-06-29' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  projects.loc[projects['Проект'] == new_project, 'Дата начала отслеживания'] = date.today()


## Алгоритм определения статуса по проекту. Определение статусов, часть 1

![Алгоритм](Алгоритм_update.drawio.png)

In [31]:
# меняем статус на в отгрузке
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Вариант обеспечения'] == 'Отгрузить'), 'Статус проектной линии'] = 'В отгрузке'

df['Дата отгрузки'] = pd.to_datetime(df['Дата отгрузки'], format="%d.%m.%Y").dt.date
shipped_goods['Дата документа РТиУ'] = pd.to_datetime(shipped_goods['Дата документа РТиУ'], yearfirst=True).dt.date
shipped_goods = shipped_goods.rename(columns={'Номер Заказа Клиента':'Номер клиентского заказа'})

# делим на 2 части (в отгрузке и нет)
df_in_shipment = df[df['Статус проектной линии'] == 'В отгрузке']
df_open = df[df['Статус проектной линии'] != 'В отгрузке']

# обрабатываем часть в отгрузке
df_in_shipment_pivot = pd.pivot_table(df_in_shipment[['Номер клиентского заказа','Артикул','Склад','Количество']], index=['Номер клиентского заказа','Артикул','Склад'], values='Количество', aggfunc='sum').reset_index()
shipped_goods_pivot = pd.pivot_table(shipped_goods[['Номер клиентского заказа','Артикул','Склад','Количество отгруженного']], index=['Номер клиентского заказа','Артикул','Склад'], values='Количество отгруженного', aggfunc='sum').reset_index()
df_in_shipment_pivot = df_in_shipment_pivot.merge(shipped_goods_pivot, on=['Номер клиентского заказа','Артикул','Склад'], how='left')
df_in_shipment_pivot['Разница'] = df_in_shipment_pivot['Количество'] - df_in_shipment_pivot['Количество отгруженного']
fully_shipped_lines = df_in_shipment_pivot[df_in_shipment_pivot['Разница'] <= 0]
df_in_shipment = df_in_shipment.merge(fully_shipped_lines[['Номер клиентского заказа','Артикул','Склад','Количество отгруженного']], how='left', on=['Номер клиентского заказа','Артикул','Склад'])
df_in_shipment.loc[(df_in_shipment['Статус проектной линии'] == 'В отгрузке') & (~df_in_shipment['Количество отгруженного'].isna()), 'Статус проектной линии'] = 'Отгружено'

# сливаем все назад
df = pd.concat([df_in_shipment, df_open])
df['Количество отгруженного'].fillna(0, inplace=True)

# меняем статус на отмена
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & ((df['Отменено'] == 'Да') | (df['Вариант обеспечения'] == 'Не обеспечивать')), 'Статус проектной линии'] = 'Отмена'

## Определение статусов по проектам, часть 2

In [33]:
# меняем статус на блок сбытовой 
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (~df['Статус продаж'].isna()) & (~df['Статус продаж'].isin(['08', '09', '10', '18', 'T1', 'T2'])), 'Статус проектной линии'] = 'Блок Сбытовой'

# проверка на статус "На складе"
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'] == 'Обеспечен на складе'), 'Статус проектной линии'] = 'На складе'

# проверка на статус "В транзите"
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'] == 'Обеспечен к дате') & (df['Заказ на поступление'].str.contains("Приобретение")), 'Статус проектной линии'] = 'В транзите'
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'] == 'Обеспечен к дате') & (df['Заказ на поступление'].str.contains("Перемещение")), 'Статус проектной линии'] = 'В транзите'

# проверка на БЛОК по закупочному статусу
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Статус закупок'] != '30') & (~df['Статус закупок'].isna()), 'Статус проектной линии'] = 'Блок Закупочный'

# проверка на наличие размещенных заказов поставщику по проектным линиям 
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'] == 'Обеспечить'), 'Статус проектной линии'] = 'Ожидается размещение РО'

# проверим, транзит с опозданием или нет 
# создадим вспомогаетльную колонку с датой для сравнения, потом удалить
df['Дата для сравнения'] = np.where(df['Желаемая дата отгрузки'] == '1900-01-01', df['Контрактная дата отгрузки'], df['Желаемая дата отгрузки'])
df['Дата для сравнения'] = pd.to_datetime(df['Дата для сравнения'], format="%d.%m.%Y").dt.date

# проверка на задержку доставки до склада по еще не отгруженным РО
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'].isin(['Обеспечен к дате', 'Ожидается по неподтвержденному заказу'])) & (df['Заказ на поступление'].str.contains("Заказ")) & (df['Дата отгрузки'] > df['Дата для сравнения']), 'Статус проектной линии'] = 'Требуется ускорение'
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'].isin(['Обеспечен к дате', 'Ожидается по неподтвержденному заказу'])) & (df['Заказ на поступление'].str.contains("озврат")) & (df['Дата отгрузки'] > df['Дата для сравнения']), 'Статус проектной линии'] = 'Требуется ускорение'
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'].isin(['Обеспечен к дате', 'Ожидается по неподтвержденному заказу'])) & (df['Заказ на поступление'].str.contains("Этап производства")) & (df['Дата отгрузки'] > df['Дата для сравнения']), 'Статус проектной линии'] = 'Требуется ускорение'
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'].isin(['Обеспечен к дате', 'Ожидается по неподтвержденному заказу'])) & (df['Заказ на поступление'].str.contains("Заказ")) & (df['Дата отгрузки'] <= df['Дата для сравнения']), 'Статус проектной линии'] = 'Ожидается отгрузка от поставщика'
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'].isin(['Обеспечен к дате', 'Ожидается по неподтвержденному заказу'])) & (df['Заказ на поступление'].str.contains("озврат")) & (df['Дата отгрузки'] <= df['Дата для сравнения']), 'Статус проектной линии'] = 'Ожидается отгрузка от поставщика'
df.loc[(df['Статус проектной линии'] == 'Статус отсутствует') & (df['Состояние обеспечения'].isin(['Обеспечен к дате', 'Ожидается по неподтвержденному заказу'])) & (df['Заказ на поступление'].str.contains("Этап производства")) & (df['Дата отгрузки'] <= df['Дата для сравнения']), 'Статус проектной линии'] = 'Ожидается отгрузка от поставщика'

#- проверить, ожидается обеспечение по неразмещенному заказу
#- ожидается утверждение заказа поставщику
# ЕЩЕ КОСМЕТИКА
# приведение формата дат
df['Дата согласования'] = pd.to_datetime(df['Дата согласования'], format='%d.%m.%Y %H:%M:%S').dt.date
df['Контрактная дата отгрузки'] = pd.to_datetime(df['Контрактная дата отгрузки'], format='%d.%m.%Y').dt.date
df['Желаемая дата отгрузки'] = pd.to_datetime(df['Желаемая дата отгрузки'], format='%d.%m.%Y').dt.date
df['Критическая дата'] = pd.to_datetime(df['Критическая дата'], yearfirst=True).dt.date
df.loc[df['Дата поступления'] != 'Поступило', 'Дата поступления'] = pd.to_datetime(df.loc[df['Дата поступления'] != 'Поступило', 'Дата поступления'], format='%d.%m.%Y').dt.date
df['Количество'] = df['Количество'].astype('int')
df['Смещение даты отгрузки (от контрактной), в днях'] = df['Дата отгрузки'] - df['Контрактная дата отгрузки']

df = df.drop(['Артикул&Склад', 'Ключ клиентского заказа'], axis=1)
df = df.rename(columns={'Проект':'Название проекта', 'Проект.Ответственный':'Лидер проекта'})

## Деление отчета по MRP контроллерам

In [36]:
# по вторникам отчет формируется, разбивается по закупащикам и сохраняется в пофамильные папке. Закупщики указывают в своих частях отчета комментарии по проектам
if today_is_Tuesday:

    if comments_needed:
        
        # выберем только те позиции, к которым нужны комменты
        df['Заказ клиента&Id строки EDI'] = df['Номер клиентского заказа'] + '%' + df['Id строки EDI'].astype('str')
        codes_for_comments_tuesday = df[df['Статус проектной линии'].isin(['Требуется ускорение', 'Ожидается размещение РО'])]['Заказ клиента&Id строки EDI'].tolist()
    
        # прикручиваем комментарии с прошлой пятницы (потому что они могут быть изменены координатором)
        
        last_friday_df = pd.read_excel(PATH + f"\\{date.today() - pd.to_timedelta(comments_delay, unit='d')}\\Выходные файлы\\Общие\\result_{date.today() - pd.to_timedelta(comments_delay, unit='d')}.xlsx")
        last_friday_df['Заказ клиента&Id строки EDI'] = last_friday_df['Номер клиентского заказа'] + '%' + last_friday_df['Id строки EDI'].astype('str')
        friday_comments = last_friday_df[(last_friday_df['Комментарий снабжения'] != '-') & (~last_friday_df['Комментарий снабжения'].isna()) & (last_friday_df['Заказ клиента&Id строки EDI'].isin(codes_for_comments_tuesday))][['Заказ клиента&Id строки EDI','Комментарий снабжения']].drop_duplicates(subset='Заказ клиента&Id строки EDI')
        df = df.merge(friday_comments, on='Заказ клиента&Id строки EDI', how='left')
        df = df.rename(columns = {'Комментарий снабжения':'Комментарий снабжения с прошлой недели'})
        
    else:
        df['Комментарий снабжения с прошлой недели'] = '-'

    df['Комментарий снабжения'] = '-'
    df.loc[df['MRP контролер'].isna(), 'MRP контролер'] = 'NO MRP (кому больше хочется, тот и заполняет)'
    df_for_controllers = df.reindex(columns=['Название проекта', 'Дата согласования', 'Артикул', 'Количество', 'Номер клиентского заказа', 'Id строки EDI', 'Заказ на поступление', "Заказ на поступление.Номер", 'Дата поступления', 'Статус проектной линии', 'Поставщик', 'Комментарий снабжения', 'Комментарий снабжения с прошлой недели',  'Склад', 'Контрактная дата отгрузки', 'Желаемая дата отгрузки', 'Дата отгрузки', 'Номенклатура', 'Характеристика', 'Дата заказа', 'Вариант обеспечения', 'Обособленно', 'Отменено', 'Частичная отгрузка', 'Критическая дата', 'Смещение даты отгрузки (от контрактной), в днях', 'MRP контролер', 'Локальный поток ДА/НЕТ', 'Статус продаж', 'Статус закупок', 'Дата для сравнения', 'Лидер проекта', 'Контрагент', 'Стоимость с НДС', 'КИС'])

    # для проверки на случай если линии были удалены
    lines_check_up = {}
    
    mrp_controllers = list(df_for_controllers['MRP контролер'].unique())
    
    for controller in mrp_controllers:
    
        result_for_controller = df_for_controllers[df_for_controllers['MRP контролер'] == controller]
        lines_check_up[controller] = len(result_for_controller)
        os.mkdir(PATH + f'\\{date.today()}\\Выходные файлы\\Снабжение\\{controller}')
    
        with pd.ExcelWriter(PATH + f"\\{date.today()}\\Выходные файлы\\Снабжение\\{controller}\\result_{date.today()}.xlsx") as writer:  
            result_for_controller.to_excel(writer, sheet_name='Данные', index=False)
            gl.to_excel(writer, sheet_name='Глоссарий', index=False) 
    
        helpers.formatting(PATH + f"\\{date.today()}\\Выходные файлы\\Снабжение\\{controller}\\result_{date.today()}.xlsx")
    
    lines = {}
    lines["Кол-во линий до комментов"] = lines_check_up
    lines_df = pd.DataFrame.from_dict(lines, orient='columns')
    lines_df.to_excel(PATH + f"\\{date.today()}\\Выходные файлы\\Линии_снабженец.xlsx")
    helpers.formatting(PATH + f"\\{date.today()}\\Выходные файлы\\Линии_снабженец.xlsx", option="purch_lines")

## Проверка на полную отгрузку проектов и их запись в архив

In [ ]:
for project in df['Название проекта'].unique():

    if len(df[(df['Название проекта'] == project) & (df['Статус проектной линии'].isin(['Отмена', 'Отгружено']))]) == len(df[df['Название проекта'] == project]):
        if projects.loc[projects['Название проекта'] == project, 'Статус проекта'].values[0] not in ['Завершен?', 'Завершен']:
            projects.loc[projects['Название проекта'] == project, 'Статус проекта'] = 'Завершен?'

if len(finished_projects['Название проекта'].unique().tolist()) != 0: # есть отгруженные проекты
    
    archive_list = pd.read_excel(PATH + '\\Постоянные файлы\\Архив.xlsx', sheet_name='Список проектов') # открываем архив
    archive_list = pd.concat([archive_list, projects.loc[projects['Статус проекта'] == 'Завершен']])

    with pd.ExcelWriter(PATH + '\\Постоянные файлы\\Архив.xlsx') as writer:  
        archive_list.to_excel(writer, sheet_name='Список проектов', index=False)

## Формирование файлов со статусами по каждому проекту

In [ ]:
# формирование отдельных файлов по каждому проекту
gl = pd.read_excel(PATH + '\\Постоянные файлы\\Глоссарий.xlsx', sheet_name='Полный')
result_for_business = df[['Проект', 'Проект.Ответственный', 'Статус проектной линии', 'Комментарий снабжения на этой неделе', 'Заказ', 'Артикул', 'Дата отгрузки', 'Id строки EDI', 'Номенклатура', 'Количество',  'Смещение даты отгрузки по сравн. с прошлой неделей', 'Дата заказа', 'Дата согласования', 'Контрактная дата отгрузки', 'Желаемая дата отгрузки', 'Контрагент', 'Склад', 'Сумма с НДС']]

result_for_client = result_for_business[['Проект', 'Проект.Ответственный', 'Статус проектной линии', 'Заказ', 'Артикул', 'Дата отгрузки', 'Id строки EDI', 'Номенклатура', 'Количество', 'Смещение даты отгрузки по сравн. с прошлой неделей', 'Дата заказа', 'Дата согласования', 'Контрактная дата отгрузки', 'Желаемая дата отгрузки', 'Контрагент', 'Склад']]
result_for_client['Склад'] = result_for_client['Склад'].apply(replace_whs)

for project in df["Проект"].unique():

    with pd.ExcelWriter(f"C:\\Users\\SEA5324\\Nextcloud\\Planning Center\\100_Projects\\06 _Project_Planning Center\\Project orders reports\\{date.today()}" + f"\\{project}_{date.today()}.xlsx") as writer:  
        result_for_business[result_for_business['Проект'] == project].to_excel(writer, sheet_name='Данные', index=False)
        # выделение строк со смещением в отдельный файл
        result_for_business[(result_for_business['Проект'] == project) & (~result_for_business['Статус проектной линии'].isin(['Отгружено', 'Отмена'])) & (result_for_business['Смещение даты отгрузки по сравн. с прошлой неделей'] > 0)].to_excel(writer, sheet_name='Строки со смещением', index=False)
        result_for_client[result_for_client['Проект'] == project].to_excel(writer, sheet_name='Файл для клиента', index=False)
        gl.to_excel(writer, sheet_name='Глоссарий', index=False)

## Формирование сводного файла со статистикой по всем проектам

In [ ]:
# СВОДНЫЙ ФАЙЛ (для руководства)
if today_is_Friday:

    df['Заказ&Артикул'] = df['Номер клиентского заказа'] + '&' + df['Артикул']
    # поменяем линии, требующие ускорения, на "На ускорении", т.к. они все в работе у отдела снабжения
    df.loc[df['Статус проектной линии'] == 'Требуется ускорение', 'Статус проектной линии'] = 'На ускорении'

    # собрать комменты от закупщиков со вторника:
    folder_names = os.listdir(PATH + f'\\{date.today() - pd.to_timedelta(comments_delay, unit='d')}\\Выходные файлы\\Снабжение')
    dfs_array = []
    
    for name in folder_names:
        df_temp = pd.read_excel(PATH + f'\\{date.today() - pd.to_timedelta(comments_delay, unit='d')}\\Выходные файлы\\Снабжение\\{name}\\result_{date.today() - pd.to_timedelta(comments_delay, unit='d')}.xlsx')
        dfs_array.append(df_temp)
    result = pd.concat(dfs_array, ignore_index=True)

    # подтянем комментарии только там, где они нужны (то есть статусы - требуется ускорение и ожидается размещение РО)
    result['Заказ клиента&Id строки EDI'] = result['Номер клиентского заказа'] + '%' + result['Id строки EDI'].astype('str') #ключ
    result = result[~result['Заказ клиента&Id строки EDI'].isna()]
    tuesday_comments = result[(result['Комментарий снабжения'] != '-') & (~result['Комментарий снабжения'].isna()) & (result['Заказ клиента&Id строки EDI'].isin(codes_for_comments))][['Заказ клиента&Id строки EDI','Комментарий снабжения']].drop_duplicates(subset='Заказ клиента&Id строки EDI')
    
    df = df.merge(tuesday_comments, on='Заказ клиента&Id строки EDI', how='left')
    col = df.pop('Комментарий снабжения')
    df.insert(4, col.name, col)

    # подсчитаем некоторые статистики для сводного файла
    array = [] #открытые линии, закрытые линии, все линии, процент отгруженного, процент своевременности 
    projects_statuses = {} # словарь с этими данными по каждому проекту
    
    for project in df['Название проекта'].unique():
        projects_statuses[project] = {}
   
        open_lines = len(df[(df['Название проекта'] == project) & (~df['Статус проектной линии'].isin(['Отгружено', 'Отмена']))]['Заказ&Артикул'].unique()) # открытые строки
        
        if open_lines == 0:
    
            projects.loc[projects['Название проекта'] == project, 'Статус проекта'] = 'Завершен?' # если нет открытых линий, то помечаем этот проект в файле как возможный завершенный
            projects.to_excel(PATH + '\\Постоянные файлы\\Projects.xlsx', index=False)
        
        projects_statuses[project]['cancelled_lines'] = len(df[(df['Название проекта'] == project) & (df['Статус проектной линии'] == 'Отмена')]['Заказ&Артикул'].unique())
        projects_statuses[project]['shipped_lines'] = len(df[(df['Название проекта'] == project) & (df['Статус проектной линии'].isin(['Отгружено']))]['Заказ&Артикул'].unique()) # для графиков
        projects_statuses[project]['not_shipped_lines'] = len(df[(df['Название проекта'] == project) & (~df['Статус проектной линии'].isin(['Отгружено', 'Отмена']))]['Заказ&Артикул'].unique()) # для графиков
        projects_statuses[project]['percent_of_shipped'] = str(round((projects_statuses[project]['shipped_lines'] / (projects_statuses[project]['shipped_lines'] + projects_statuses[project]['not_shipped_lines'])) * 100)) + '%'
        projects_statuses[project]['delayed_lines_contract'] = len(df[(df['Дата отгрузки'] > df['Контрактная дата отгрузки']) & (df['Статус проектной линии'] != 'Отмена') & (df['Название проекта'] == project)]['Заказ&Артикул'].unique())
        projects_statuses[project]['delayed_lines_desired'] = len(df[(df['Дата отгрузки'] > df['Желаемая дата отгрузки']) & (df['Статус проектной линии'] != 'Отмена') & (df['Название проекта'] == project)]['Заказ&Артикул'].unique())
        projects_statuses[project]['not_delayed_lines_desired'] = len(df[(df['Дата отгрузки'] <= df['Желаемая дата отгрузки']) & (df['Статус проектной линии'] != 'Отмена') & (df['Название проекта'] == project)]['Заказ&Артикул'].unique())
        projects_statuses[project]['not_delayed_lines_contract'] = len(df[(df['Дата отгрузки'] <= df['Контрактная дата отгрузки']) & (df['Статус проектной линии'] != 'Отмена') & (df['Название проекта'] == project)]['Заказ&Артикул'].unique())
        projects_statuses[project]['all_lines'] = len(df[df['Название проекта'] == project]['Заказ&Артикул'].unique())
        projects_statuses[project]['percent_of_not_delayed_contract'] = str(round((projects_statuses[project]['not_delayed_lines_contract'] / (projects_statuses[project]['not_delayed_lines_contract'] + projects_statuses[project]['delayed_lines_contract'])) * 100)) + '%'
        projects_statuses[project]['percent_of_not_delayed_desired'] = str(round((projects_statuses[project]['not_delayed_lines_desired'] / (projects_statuses[project]['not_delayed_lines_desired'] + projects_statuses[project]['delayed_lines_desired'])) * 100)) + '%'
        projects_statuses[project]['project_leader'] = projects[projects['Название проекта'] == project]['Лидер проекта'].values[0]
        projects_statuses[project]['bitrix_number'] = projects[projects['Название проекта'] == project]['Номер сделки'].values[0]
        projects_statuses[project]['end_user'] = projects[projects['Название проекта'] == project]['Конечный заказчик'].values[0]
        projects_statuses[project]['project_value'] = df[(df['Статус проектной линии'] != 'Отмена') & (df['Название проекта'] == project)]['Сумма с НДС'].sum()
        array.append([project, projects_statuses[project]['end_user'], projects_statuses[project]['project_leader'], projects_statuses[project]['cancelled_lines'], projects_statuses[project]['not_shipped_lines'], projects_statuses[project]['shipped_lines'], projects_statuses[project]['all_lines'], projects_statuses[project]['percent_of_shipped'], projects_statuses[project]['percent_of_not_delayed_contract'], projects_statuses[project]['percent_of_not_delayed_desired'], projects_statuses[project]['project_value'], projects_statuses[project]['bitrix_number']])    
             
    columns_names = ['Проект', 'Конечный заказчик', 'Лидер проекта', 'Отменено', 'К поставке', 'Поставлено', 'Всего', f'Отгружено на {date.today()}', 'Вовремя (контр.)', 'Вовремя (жел.)', 'Сумма, RUB', 'Номер сделки']  
    general_pivot = pd.DataFrame(array, columns=columns_names)
    
    # загрузка сводного файла с прошлой недели для сравнения
    
    pivot_last_week = pd.read_excel(PATH + f'\\{date.today() - pd.to_timedelta(last_week_delay, unit='d')}\\Выходные файлы\\Общие\\result_{date.today() - pd.to_timedelta(last_week_delay, unit='d')}.xlsx', sheet_name='Сводный')
    comments_last_week = pd.read_excel(PATH + f'\\{date.today() - pd.to_timedelta(last_week_delay, unit='d')}\\Выходные файлы\\Общие\\result_{date.today() - pd.to_timedelta(last_week_delay, unit='d')}.xlsx', sheet_name='Комментарии')
    general_pivot = general_pivot.merge(pivot_last_week[['Проект', f'Отгружено на {date.today() - pd.to_timedelta(last_week_delay, unit='d')}']], on='Проект', how='left')
    comments_df = general_pivot[['Проект']].merge(comments_last_week[['Проект', f'Комментарий на {date.today() - pd.to_timedelta(last_week_delay, unit='d')}']], on='Проект', how='left')
    comments_df[f'Комментарий на {date.today()}'] = '-'
    col = general_pivot.pop(f'Отгружено на {date.today() - pd.to_timedelta(last_week_delay, unit='d')}')
    general_pivot.insert(7, col.name, col)
    general_pivot = general_pivot.rename(columns={'Проект':'Проект (#00ac86)'}) # код корпоративного цвета
   
    # ключ в новой таблице
    df['Заказ клиента&Id строки EDI'] = df['Номер клиентского заказа'] + '%' + df['Id строки EDI'].astype('str')
    # строки, к которым нужны комменты
    codes_for_comments = df[df['Статус проектной линии'].isin(['На ускорении', 'Ожидается размещение РО'])]['Заказ клиента&Id строки EDI'].unique().tolist()

In [ ]:
# формирование общего файла для координатора проектов со сводным файлом для отправки руководству (только по пятницам)
df_for_coordinator = df.reindex(columns=['Название проекта', 'Лидер проекта', 'Статус проектной линии', 'Причина проблемы', 'Дата отгрузки', 'Номер клиентского заказа', 'Артикул', 'Количество', 'Id строки EDI', 'Поставщик', 'Комментарий снабжения', 'Комментарий снабжения с прошлой недели', 'Контрактная дата отгрузки', 'Частично обеспечено', 'Частично не обеспечено', 'MRP контролер', 'Расчетная дата отгрузки транзита', 'Клиент.Код', 'Номенклатура',  'Желаемая дата отгрузки', 'Критическая дата', 'Склад', 'Заказ на поступление', "Заказ на поступление.Номер", 'Дата поступления', 'Дата согласования',  'Дата заказа', 'Вариант обеспечения', 'Обособленно', 'Отменено',  'Смещение даты отгрузки (от контрактной), в днях', 'Локальный поток ДА/НЕТ',  'Статус продаж','Статус закупок', 'Дата для сравнения', 'Контрагент', 'Стоимость с НДС', 'КИС', 'Наличие у дистров', 'Остаток РС', 'Остаток Элевел', 'Остаток ЭТМ'])
with pd.ExcelWriter(PATH + f"\\{date.today()}\\Выходные файлы\\Общие\\result_{date.today()}.xlsx") as writer:  
    df_for_coordinator.to_excel(writer, sheet_name='Данные', index=False)
    gl1.to_excel(writer, sheet_name='Глоссарий', index=False)
    if today_is_Friday:
        general_pivot.to_excel(writer, sheet_name='Сводный', index=False)
        comments_df.to_excel(writer, sheet_name='Комментарии', index=False)

helpers.formatting(PATH + f"\\{date.today()}\\Выходные файлы\\Общие\\result_{date.today()}.xlsx")

## Автоматическая рассылка отчетов

In [9]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

In [10]:
# Настройки для отправки почты
smtp_server = 'mail.systeme.ru'  # SMTP сервер
smtp_port = 587  # Порт для TLS
smtp_user = 'SEA5324'#Нужно ввести свой
smtp_password = 'yhzL4cdyg6' #Нужно ввести свой
from_email = 'irina.perepelkina@systeme.ru' #Нужно ввести свой корпоративный

In [12]:
projects = pd.read_excel(PATH + '\\Постоянные файлы\\Projects.xlsx')
emails = projects[['Проект', 'Email']].drop_duplicates(subset='Проект')

In [11]:
# Функция для отправки письма с вложением
def send_email(to_email, subject, body, file_path):
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # Вложение
    if os.path.isfile(file_path):
        file_name = os.path.basename(file_path)
        content_type = 'application/octet-stream'  # Общее значение, если не определено иначе

        # Определение типа содержимого файла по расширению
        if file_name.endswith('.xlsx'):
            content_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        elif file_name.endswith('.xls'):
            content_type = 'application/vnd.ms-excel'

        with open(file_path, 'rb') as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header(
                'Content-Disposition',
                'attachment', filename=file_name,
            )
            part.add_header('Content-Type', content_type)
            msg.attach(part)
    else:
        print(f"Файл не найден: {file_path}")
        return

    # Отправка сообщения
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # Защита соединения
        server.login(smtp_user, smtp_password)
        server.send_message(msg)
        print(f"Письмо отправлено на {to_email}")

In [ ]:
#Обработка строк в Excel и отправка писем
for index, row in emails.iterrows():
    file_name = row['Проект'] + f'_{date.today()}' + '.xlsx'
    recipient_email = row['Email']
    file_path = os.path.join(f'C:\\Users\\SEA5324\\Nextcloud\\Planning Center\\100_Projects\\06 _Project_Planning Center\\Project orders reports\\{date.today()}\\', file_name) #Указать правильный путь к файлам    
    send_email(
        to_email=recipient_email,
        subject=f'Отслеживание проектов {date.today()}: {row['Проект']}',
        body='Добрый день, коллеги!\n\nВо вложении обновление по проекту.\n\nЕсли есть какие-то вопросы или нужна дополнительная проработка по проекту, пожалуйста, пишите.\n\nКоординатор проектов,\nПерепелкина Ирина',
        file_path=file_path,
    )

## Оценщик сложности проектов

In [67]:
finished_projects = finished_projects[finished_projects['Отменено'] == 'Нет'] # оцениваем только линии без отмен

if len(finished_projects) != 0:
    
    finished_projects['Заказ клиента&Id строки EDI'] = finished_projects['Номер клиентского заказа'] + '%' + finished_projects['Id строки EDI'].astype(str)
    evaluator_file_path = PATH + "\\Постоянные файлы\\Оценщик_сложности.xlsx"
    # Загружаем файл какой он сейчас есть, потом к нему прибавим новые строки и сохраним
    evaluator_df = pd.read_excel(evaluator_file_path)

    ABC_RU01 = pd.read_excel(PATH + '\\Постоянные файлы\\ABC-FMR.xlsx', sheet_name = 'Final RU01')
    ABC_RU06 = pd.read_excel(PATH + '\\Постоянные файлы\\ABC-FMR.xlsx', sheet_name = 'Final RU06')
    sales_RU01 = pd.read_excel(PATH + '\\Постоянные файлы\\ABC-FMR.xlsx', sheet_name = 'Sales RU01', skipfooter=2)[['Reference', 'Amu']]
    sales_RU06 = pd.read_excel(PATH + '\\Постоянные файлы\\ABC-FMR.xlsx', sheet_name = 'Sales RU06', skipfooter=2)[['Reference', 'Amu']]

    # подготовка ключей
    ABC_RU01['Артикул&Склад'] = ABC_RU01['Reference'] + '&RU01'
    ABC_RU06['Артикул&Склад'] = ABC_RU06['Reference'] + '&RU06'
    ABC_TOTAL = pd.concat([ABC_RU01, ABC_RU06])
    finished_projects['Склад_коротко'] = finished_projects['Склад'].apply(change_whs)
    finished_projects['Артикул&Склад'] = finished_projects['Артикул'] + '&' + finished_projects['Склад_коротко']
    finished_projects = finished_projects.merge(ABC_TOTAL[['Category', 'Артикул&Склад']], on='Артикул&Склад', how='left')
    
    sales_RU01['Артикул&Склад'] = sales_RU01['Reference'] + '&RU01'
    sales_RU06['Артикул&Склад'] = sales_RU06['Reference'] + '&RU06'
    sales = pd.concat([sales_RU01, sales_RU06])
    finished_projects = finished_projects.merge(sales[['Amu', 'Артикул&Склад']], on='Артикул&Склад', how='left')
    для данных, которые рассчитываются не автоматически
    add_resoures_and_fc = projects[projects['Проект'].isin(finished_projects['Название проекта'].unique())].drop_duplicates(subset='Проект')

    for project in list(finished_projects['Название проекта'].unique()):
    
        print(project)
        # какие данные понадобятся для датафрейма
        tight_deadlines = 0
        rare_refs = 0
        el_order = 0
        suppliers_qty = 0
        whs_qty = 0
        prj_duration = 0
    
        all_lines = len(finished_projects[finished_projects['Название проекта'] == project].drop_duplicates(subset='Заказ клиента&Id строки EDI'))
    
        tight_deadlines = metrics.tight_lines_evaluator(project, all_lines, finished_projects)
        rare_refs = metrics.rare_refs_evaluator(project, all_lines, finished_projects)
        el_order = metrics.el_orders_evaluator(project, all_lines, finished_projects)
        suppliers_qty = len(finished_projects[finished_projects['Название проекта'] == project]['Поставщик'].unique())
        suppliers_points = metrics.suppliers_evaluator(suppliers_qty)
        whs_to_pass = finished_projects[finished_projects['Название проекта'] == project]['Склад'].unique()
        whs_points = metrics.whs_evaluator(whs_to_pass)
        lines_points = metrics.lines_qty_evaluator(all_lines)
        prj_duration = (max(pd.to_datetime(finished_projects[finished_projects['Название проекта'] == project]['Дата отгрузки'], format='%d.%m.%Y').dt.date) - min(pd.to_datetime(finished_projects[finished_projects['Название проекта'] == project]['Дата заказа'], format='%d.%m.%Y').dt.date)).days
        prj_duration_points = 0 if prj_duration < 180 else 1
        project_value = finished_projects[finished_projects['Название проекта'] == project].drop_duplicates(subset='Заказ клиента&Id строки EDI')['Сумма с НДС'].sum()
    
        add_resources = add_resoures_and_fc[add_resoures_and_fc['Проект'] == project]['Доп ресурсы'].values[0]
        sum_of_points = sum([tight_deadlines, rare_refs, el_order, suppliers_points, whs_points, lines_points, prj_duration_points, add_resources])
        category = metrics.define_category(sum_of_points)
        
        # создаем датафрейм для записи в файл
        new_df = pd.DataFrame(columns=evaluator_df.columns, data=[[project, project_value, tight_deadlines, rare_refs, el_order, suppliers_points, whs_points, lines_points, prj_duration_points, add_resources, sum_of_points, category]])
        evaluator_df = pd.concat([evaluator_df, new_df])
        evaluator_df.to_excel(evaluator_file_path, index=False)
    
    projects['Дата заказа'] = pd.to_datetime(projects['Дата заказа'], yearfirst=True).dt.date
    projects['Критическая дата'] = pd.to_datetime(projects['Критическая дата'], yearfirst=True).dt.date
    # убираем эти строки из списка проектов
    finished_projects.to_excel(PATH + "\\finished.xlsx", index=False)
projects.to_excel(PATH + '\\Постоянные файлы\\Projects.xlsx', index=False)

#pl_Пороховой завод
